In [1]:
# Imports

import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split

import torchvision
import torchvision.transforms as transforms

from sklearn.metrics import classification_report

In [2]:
# Define the network

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # First convolutional layer: 3 input channels (RGB), outputs 16 channels
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        # Flatten output of the convolutional layers
        x = x.view(-1, 64 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# Create instance
model = Net()
print(model)

# Random data for testing
random_test = torch.rand((1, 3, 32, 32))

result = model(random_test)
print(result)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
tensor([[-0.0874,  0.0153, -0.0624,  0.0414, -0.0396, -0.0955, -0.0510,  0.0430,
         -0.0859, -0.0748]], grad_fn=<AddmmBackward0>)


In [7]:
# Definitions
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10
TRAIN_SPLIT = 0.75
VAR_SPLIT = 1 - TRAIN_SPLIT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Transformation?

# Load data

fullTrainingSet = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transforms.ToTensor())

# We'll split our training set into 80% training, 20% validation set

trainingSize = int(0.8 * len(fullTrainingSet))
valSize = len(fullTrainingSet) - trainingSize
trainset, valset = torch.utils.data.random_split(fullTrainingSet, [trainingSize, valSize])

# Load the test set
testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transforms.ToTensor())

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True)
testloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=False)

trainSteps = len(trainloader.dataset) // BATCH_SIZE
testSteps = len(testloader.dataset) // BATCH_SIZE

Files already downloaded and verified
Files already downloaded and verified


In [8]:
# More parameters

opt = torch.optim.Adam(model.parameters(), lr=INIT_LR)

lossF = nn.CrossEntropyLoss()

H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}

print("Starting training...")
startTime = time.time()

for e in range(0, EPOCHS):
    print(f"Starting epoch {e+1}/{EPOCHS}")
    model.train()

    totalTrainLoss = 0
    totalValLoss = 0

    trainCorrect = 0
    valCorrect = 0

    for (x, y) in trainloader:
        (x, y) = (x.to(device), y.to(device))

        pred = model(x)
        loss = lossF(pred, y)

        opt.zero_grad()
        loss.backward()
        opt.step()

        totalTrainLoss += loss
        trainCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()

    # Validation
    with torch.no_grad():
        model.eval()

        for (x, y) in valloader:
            (x, y) = (x.to(device), y.to(device))
            
            pred = model(x)
            
            totalValLoss += lossF(pred, y)
            valCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()

    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / testSteps

    trainCorrect /= len(trainloader.dataset)
    valCorrect /= len(testloader.dataset)

    # Todo: Update history

    print(f"Epoch {e + 1} complete\nLoss: {avgTrainLoss}\nAccuracy: {trainCorrect}\nValidation loss: {avgValLoss}\nValidation accuracy: {valCorrect}\n")

endTime = time.time()

print(f"Training finished in {endTime-startTime}s, starting validation")

preds = []
targets = []

with torch.no_grad():
    model.eval()

    

    for (image, label) in testloader:
        image = image.to(device)
        
        pred = model(image)
        preds.extend(pred.argmax(axis=1).cpu().numpy())
        targets.extend(label.cpu().numpy())

print(classification_report(targets, preds, target_names=testset.classes))

Starting training...
Starting epoch 1/10
Epoch 1 complete
Loss: 0.8791881799697876
Accuracy: 0.68955
Validation loss: 0.2335376739501953
Validation accuracy: 0.168425

Starting epoch 2/10
Epoch 2 complete
Loss: 0.8223723769187927
Accuracy: 0.71065
Validation loss: 0.22199738025665283
Validation accuracy: 0.172625

Starting epoch 3/10
Epoch 3 complete
Loss: 0.7705252170562744
Accuracy: 0.727875
Validation loss: 0.22818176448345184
Validation accuracy: 0.17055

Starting epoch 4/10
Epoch 4 complete
Loss: 0.7259247303009033
Accuracy: 0.745
Validation loss: 0.22352515161037445
Validation accuracy: 0.17225

Starting epoch 5/10
Epoch 5 complete
Loss: 0.6825100779533386
Accuracy: 0.761425
Validation loss: 0.213173970580101
Validation accuracy: 0.17715

Starting epoch 6/10
Epoch 6 complete
Loss: 0.651515543460846
Accuracy: 0.770125
Validation loss: 0.21549057960510254
Validation accuracy: 0.1775

Starting epoch 7/10
Epoch 7 complete
Loss: 0.6114392876625061
Accuracy: 0.785075
Validation loss: 0